In [1]:
trainingOHCA_fileName = "generatedPoints1_100000_area.csv"
candidateAED_fileName = "buildings_locations_area.csv"

In [2]:
import pandas as pd
import numpy as np
from haversine import haversine_vector, Unit
from haversine import haversine
from os import path
import scipy.sparse
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum
from pulp import *
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt     
import seaborn as sns
from itertools import combinations
from sklearn import preprocessing, cluster
from sklearn.metrics import silhouette_score
import scipy
import scipy.cluster
from matplotlib import colors as mcolors
from skimage.io import imread
import math
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
import time

In [4]:
def MCLP(S,OHCA_Cluster,AED_Cluster,d,P): #Input for MCLP to return the coordinates of AEDs to be placed
    
    # S is Distance beyond which is considered uncovered
    # P is the number of AEDs to be placed in that cluster 
    
    I = OHCA_Cluster.index.values.tolist() #Demand Node O(N)
    J = AED_Cluster.index.values.tolist() #Facility Node O(N)
    P = P
    #a = np.ones(len(OHCA_Cluster))
    
    # N is a list in which each item is a list of nodes within threshold distance of ith node O(N^2)
    N = [[j for j in J if d[i][j] < S] for i in I]
    #P = 2400 #Number of AEDs to be placed in this cluster
    
    #Optimization                             ###Here she specifies the LP variables
    prob = LpProblem("MCLP",LpMaximize)
    x = LpVariable.dicts("x",J,0,1,cat="Integer")
    y = LpVariable.dicts("y",I,0,1,cat="Integer")
    #a = LpVariable.dicts("a",I,cat="Continuous")
    
    #Objective                               ### Calculate the sum of a list of linear expressions O(N)
    #prob += lpSum([a[i]*y[i] for i in I])
    prob += lpSum([y[i] for i in I])
    
    #Constraints
    for i in I:
        prob += lpSum([x[j] for j in N[i]]) >= y[i]
       # for j in N[i]:
            #if x[j] == 1:
                #prob += a[i] == 0.549*d[i][j]**-0.584
        #a[i] = [0.549*d[i][j]**-0.584 for j in N[i] if x[j] == 1]
        
    prob +=lpSum([x[j] for j in J]) == P

    #Solving
    #solver = getSolver('GUROBI')
    #prob.solve(solver)
    prob.solve(GUROBI(msg=0))
    
    x_soln = np.array([x[j].varValue for j in J])
    print ("OHCA Served is = ", value(prob.objective))
    AED_Chosen = pd.concat([AED_Cluster,pd.DataFrame(x_soln,columns=['x'])],axis=1)
    AED_Chosen = AED_Chosen[AED_Chosen['x'] == 1]
    AED_Chosen = AED_Chosen.drop(columns=['x'])
    return(AED_Chosen)

In [5]:
#test_OHCA = pd.read_csv(testOHCA_fileName, names = ["LatOHCA", "LongOHCA"], header = None)
OHCA = pd.read_csv(trainingOHCA_fileName)
candidateAED = pd.read_csv(candidateAED_fileName)
# candidateAED.columns = ["LatAED","LongAED"]
P_count=OHCA['region'].value_counts()
type(P_count)

pandas.core.series.Series

In [6]:
def haverDist(OHCA_Cluster,AED_Cluster):
    d = haversine_vector(list(zip(OHCA_Cluster.LatOHCA, OHCA_Cluster.LongOHCA)), list(zip(AED_Cluster.LatAED, AED_Cluster.LongAED)), Unit.METERS, comb=True).T
    return(d)

In [7]:
%%time

start_time = time.time()

ALL_AED= pd.DataFrame(columns=['LatAED', 'LongAED']) 
all_AED_num=9880
all_OHCA_num=97542

for subzone in range(1,56):
    if subzone != 20 and subzone!=14 and subzone != 48 : 
        p_subzone=int(P_count.at[subzone]*9880/97542)
        print(subzone, P_count.at[subzone], p_subzone )
        OHCA1=OHCA[OHCA['region']== subzone]
        OHCA1=OHCA1[['LatOHCA','LongOHCA']].reset_index(drop=True)
        candidateAED1 = candidateAED[candidateAED['region']==subzone].copy()
        candidateAED1 = candidateAED1[['LatAED','LongAED']].reset_index(drop=True)
        candidateAED1 = candidateAED1.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()
        d1 = haverDist(OHCA1,candidateAED1)
        sub_AED_df = MCLP(100,OHCA1,candidateAED1,d1,p_subzone)
        ALL_AED=ALL_AED.append(sub_AED_df)
        
print("Time is --- %s seconds ---" % (time.time() - start_time))
ALL_AED

1 4588 460
Academic license - for non-commercial use only - expires 2022-06-25
Using license file C:\Users\ISEdzh\gurobi.lic
OHCA Served is =  3452.0
2 1586 159
OHCA Served is =  1184.0
3 1932 194
OHCA Served is =  1280.0
4 1960 196
OHCA Served is =  None
5 1123 112
OHCA Served is =  350.0
6 1476 148
OHCA Served is =  1167.0
7 1796 180
OHCA Served is =  1315.0
8 4285 430
OHCA Served is =  3801.0
9 2791 280
OHCA Served is =  1545.0
10 3835 385
OHCA Served is =  3187.0
11 3245 326
OHCA Served is =  2734.0
12 97 9
OHCA Served is =  11.0
13 1793 180
OHCA Served is =  94.0
15 2113 212
OHCA Served is =  1794.0
16 2377 238
OHCA Served is =  1520.0
17 312 31
OHCA Served is =  100.0
18 1198 120
OHCA Served is =  765.0
19 3318 333
OHCA Served is =  2562.0
21 1947 195
OHCA Served is =  1472.0
22 394 39
OHCA Served is =  172.0
23 2794 280
OHCA Served is =  1889.0
24 3104 311
OHCA Served is =  2254.0
25 509 51
OHCA Served is =  107.0
26 1616 162
OHCA Served is =  1112.0
27 2705 271
OHCA Served is =

,LatAED,LongAED
0,1.278086,103.837117
3,1.279740,103.839514
5,1.276883,103.840086
12,1.274570,103.839212
24,1.274428,103.834879
...,...,...
3011,1.400427,103.822481
3048,1.396961,103.835141
3052,1.400692,103.854979
3053,1.409627,103.858436


In [73]:
# Time is --- 440.95172810554504 seconds ---
# Wall time: 7min 20s

# and subzone 20 run time is 

# Time is --- 63.71788048744202 seconds -

The overall MCLP AED result is in this dataframe : 


,LatAED,LongAED
0,1.278086,103.837117
3,1.279740,103.839514
5,1.276883,103.840086
12,1.274570,103.839212
24,1.274428,103.834879
...,...,...
3011,1.400427,103.822481
3048,1.396961,103.835141
3052,1.400692,103.854979
3053,1.409627,103.858436


In [10]:
##special consideration for planning area 20

# %%time
start_time = time.time()

subzone= 20
p_subzone=int(P_count.at[subzone]*9880/97542)
print(subzone, P_count.at[subzone], p_subzone )
OHCA1=OHCA[OHCA['region']== 20]
OHCA1=OHCA1[['LatOHCA','LongOHCA']].reset_index(drop=True)

OHCA20a= OHCA1[OHCA1['LatOHCA']<= 1.324671]
OHCA20b= OHCA1[OHCA1['LatOHCA']> 1.324671]

OHCA1.describe()
# OHCA20a
OHCA20a.reset_index(drop=True,inplace=True)
OHCA20b.reset_index(drop=True,inplace=True)
candidateAED1 = candidateAED[candidateAED['region']==subzone].copy()
candidateAED1 = candidateAED1[['LatAED','LongAED']].reset_index(drop=True)
candidateAED1 = candidateAED1.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()
candidateAED20a= candidateAED1[candidateAED1['LatAED']<= 1.324671]
candidateAED20b= candidateAED1[candidateAED1['LongAED']> 1.324671]
candidateAED20a.reset_index(drop=True,inplace=True)
candidateAED20b.reset_index(drop=True,inplace=True)
OHCA20a

d1 = haverDist(OHCA20a,candidateAED20a)
sub_AED_df = MCLP(100,OHCA20a,candidateAED20a,d1,p_subzone/2)
ALL_AED=ALL_AED.append(sub_AED_df)
# uncomment this when re-run
d1 = haverDist(OHCA20b,candidateAED20b)
OHCA20b.reset_index(drop=True,inplace=True)
candidateAED20b.reset_index(drop=True,inplace=True)
sub_AED_df = MCLP(100,OHCA20b,candidateAED20b,d1,p_subzone/2)
ALL_AED=ALL_AED.append(sub_AED_df)
sub_AED_df

print("Time is --- %s seconds ---" % (time.time() - start_time))
ALL_AED

20 7528 756
OHCA Served is =  2987.0
OHCA Served is =  3340.0
Time is --- 63.71788048744202 seconds ---


,LatAED,LongAED
0,1.278086,103.837117
3,1.279740,103.839514
5,1.276883,103.840086
12,1.274570,103.839212
24,1.274428,103.834879
...,...,...
16767,1.327854,103.951251
16821,1.328017,103.952696
16847,1.326507,103.953711
16875,1.325381,103.954333


In [11]:
print('The final MCLP AED result including all areas is in this dataframe, you can use it to further calculate metrics : ')
ALL_AED

The final MCLP AED result including all areas is in this dataframe, you can use it to further calculate metrics : 


,LatAED,LongAED
0,1.278086,103.837117
3,1.279740,103.839514
5,1.276883,103.840086
12,1.274570,103.839212
24,1.274428,103.834879
...,...,...
16767,1.327854,103.951251
16821,1.328017,103.952696
16847,1.326507,103.953711
16875,1.325381,103.954333


In [13]:
def totalCoverage(ohca_df, aed_df, MAX_DIST_METERS = 100):
    def ClosestAED(r):
        # Cartesian Distance: square root of (x2-x1)^2 + (y2-y1)^2
        distances = ((r['LatOHCA']-aed_df['LatAED'])**2 + (r['LongOHCA']-aed_df['LongAED'])**2)**0.5

        # AED with minimum Distance from the OHCA
        closestAEDId = distances[distances == distances.min()].index.to_list()[0]
        return aed_df.loc[closestAEDId, ['LatAED', 'LongAED']]

    cp_ohca = ohca_df.copy()
    cp_ohca[['ClosestAEDLatitude', 'ClosestAEDLongtitude']] = cp_ohca.apply(ClosestAED, axis=1)
    
    def haversine_np(lat1, lon1, lat2, lon2):
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6367 * c
        return km * 1000
    
    cp_ohca['dist_nearest_AED'] = haversine_np(cp_ohca['LatOHCA'], cp_ohca['LongOHCA'],
                                               cp_ohca['ClosestAEDLatitude'], cp_ohca['ClosestAEDLongtitude'])
    # distance to nearest AED (in metres) is recorded for each OHCA
    
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] <= 100, 'isCovered'] = 'covered'
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] > 100,'isCovered'] = 'uncovered'
    # each OHCA having an AED within 100m has isCovered = 'covered', otherwise 'uncovered'
    #print(cp_ohca)
    
    
    within_100m  = sum(cp_ohca['isCovered'] == 'covered')
    outside_100m = len(cp_ohca) - within_100m
    total_coverage_within_100m = within_100m / (within_100m + outside_100m)
    print("total coverage = ", total_coverage_within_100m)
    
    return total_coverage_within_100m

def partial_coverage(ohca_df, aed_df, ALPHA = 0.05):
    def ClosestAED(r):
        # Cartesian Distance: square root of (x2-x1)^2 + (y2-y1)^2
        distances = ((r['LatOHCA']-aed_df['LatAED'])**2 + (r['LongOHCA']-aed_df['LongAED'])**2)**0.5

        # AED with minimum Distance from the OHCA
        closestAEDId = distances[distances == distances.min()].index.to_list()[0]
        return aed_df.loc[closestAEDId, ['LatAED', 'LongAED']]

    cp_ohca = ohca_df.copy()
    cp_ohca[['ClosestAEDLatitude', 'ClosestAEDLongtitude']] = cp_ohca.apply(ClosestAED, axis=1)
    
    def haversine_np(lat1, lon1, lat2, lon2):
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6367 * c
        return km * 1000

    cp_ohca['dist_nearest_AED'] = haversine_np(cp_ohca['LatOHCA'], cp_ohca['LongOHCA'],
                                               cp_ohca['ClosestAEDLatitude'], cp_ohca['ClosestAEDLongtitude'])
    # distance to nearest AED (in metres) is recorded for each OHCA
    
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] <= 20,'p_ij_coverage'] = 1.0
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] >= 100,'p_ij_coverage'] = 0.0
    cp_ohca.loc[(cp_ohca['dist_nearest_AED'] < 100) & (cp_ohca['dist_nearest_AED'] > 20),
                'p_ij_coverage'] = np.exp(- ALPHA * (cp_ohca['dist_nearest_AED'] - 20))
    # p_ij of each OHCA calulated
    # print(cp_ohca)
    
    return cp_ohca['p_ij_coverage'].mean()

def expectedsurvival(OHCA,AED):
    prob = 0
    for i in range(len(OHCA)):
        survival = haversine_vector((OHCA.LatOHCA[i], OHCA.LongOHCA[i]), list(zip(AED.LatAED, AED.LongAED)), Unit.METERS, comb=True).T[0]
        survival = survival/(6.15*1000/60)*2
        survival = [p if np.isnan(p) == False else 0 for p in survival]
        survival = [1 if p<=1 and p != 0 else p for p in survival]
        survival = [p if p<20 else 0 for p in survival]
        survival = [p if p==0 or p == 1 else ((p**-0.584)*0.549) for p in survival]
        prob += max(survival)
    expectedSurvival = prob/len(OHCA)
    return(expectedSurvival)

In [14]:
def average_distance(ohca_df, aed_df):
    def ClosestAED(r):
        # Cartesian Distance: square root of (x2-x1)^2 + (y2-y1)^2
        distances = ((r['LatOHCA']-aed_df['LatAED'])**2 + (r['LongOHCA']-aed_df['LongAED'])**2)**0.5

        # AED with minimum Distance from the OHCA
        closestAEDId = distances[distances == distances.min()].index.to_list()[0]
        return aed_df.loc[closestAEDId, ['LatAED', 'LongAED']]

    cp_ohca = ohca_df.copy()
    cp_ohca[['ClosestAEDLatitude', 'ClosestAEDLongtitude']] = cp_ohca.apply(ClosestAED, axis=1)
    
    def haversine_np(lat1, lon1, lat2, lon2):
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6367 * c
        return km * 1000

    cp_ohca['dist_nearest_AED'] = haversine_np(cp_ohca['LatOHCA'], cp_ohca['LongOHCA'],
                                               cp_ohca['ClosestAEDLatitude'], cp_ohca['ClosestAEDLongtitude'])
    # distance to nearest AED (in metres) is recorded for each OHCA
    
    
    return cp_ohca['dist_nearest_AED'].mean()

In [15]:
AED_Chosen_All_MCLP=ALL_AED.reset_index(drop=True)
#AED_Chosen_All_MCLP = pd.read_csv("MCLP_SingleBest_Output.csv",index_col=0)
#total coverage 
MCLP_TC = totalCoverage(OHCA,AED_Chosen_All_MCLP,100)
print("MCLP total coverage is : ", MCLP_TC)
#partial coverage
MCLP_PC = partial_coverage(OHCA,AED_Chosen_All_MCLP)
print("MCLP partial coverage is : ", MCLP_PC)
#expected survival
MCLP_ES = expectedsurvival(OHCA,AED_Chosen_All_MCLP)
print("MCLP expected survival is : ", MCLP_ES)
#average distance to nearest AED
MCLP_AD = average_distance(OHCA, AED_Chosen_All_MCLP)
print("MCLP average distance is : ", MCLP_AD)

total coverage =  0.6912887913800351
MCLP total coverage is :  0.6912887913800351
MCLP partial coverage is :  0.15479602489098832
MCLP expected survival is :  0.4954499646697673
MCLP average distance is :  161.5120564656188
